# Avoid Causing State Transations in Generators with thros

In [1]:
class MyError(Exception):
    pass

In [2]:
def my_generator():
    yield 1
    yield 2
    yield 3

In [3]:
it = my_generator()
print(next(it)) # Yield 1
print(next(it)) # Yield 2
print(it.throw(MyError('test error')))

1
2


MyError: test error

In [6]:
def my_generator():
    yield 1
    try:
        yield 2
    except MyError:
        print('Got MyError!')
    else:
        yield 3
    yield 4

In [7]:
it = my_generator()
print(next(it)) # Yield 1
print(next(it)) # Yield 2
print(it.throw(MyError('test error')))

1
2
Got MyError!
4


In [8]:
class Reset(Exception):
    pass

In [9]:
def timer(period):
    current = period
    while current:
        current -= 1
        try:
            yield current
        except Reset:
            current = period

In [10]:
def check_for_reset():
    # Poll for external event
    ...

In [11]:
def announce(remaining):
    print(f'{remaining} ticks remaining')

In [13]:
def run():
    it = timer(4)
    while True:
        try:
            if check_for_reset():
                current = it.throw(Reset())
            else:
                current = next(it)
        except StopIteration:
            break
        else:
            announce(current)

In [14]:
run()

3 ticks remaining
2 ticks remaining
1 ticks remaining
0 ticks remaining


In [15]:
class Timer:
    def __init__(self, period):
        self.current = period
        self.period = period
    def reset(self):
        self.current = self.period
    def __iter__(self):
        while self.current:
            self.current -= 1
            yield self.current

In [16]:
def run():
    timer = Timer(4)
    for current in timer:
        if check_for_reset():
            timer.reset()
        announce(current)

In [17]:
run()

3 ticks remaining
2 ticks remaining
1 ticks remaining
0 ticks remaining
